In [5]:
# -*- coding: utf-8 -*-
"""
@author: Marion
"""
# =============================================================================
# Partie SNCF
# =============================================================================
import pandas as pd
import requests
from datetime import datetime

#clé pour accéder à l'API
token_auth = '1ae705b6-570c-4652-8562-346983b6f776'


#création de fonction pour donner la date et l'heure de départ
def page_gares(numero_page):
    headers = {
        "Authorization": f"Bearer {token_auth}"
    }

    return requests.get(
        ('https://api.sncf.com/v1/coverage/sncf/journeys').format(numero_page),
        headers=headers
    )

# fonction pour convertir en date la chaine de caractères de l'API 
#et inversement, pour convertir la chaîne de caractère en date
def convertir_en_temps(chaine) :
    ''' on convertit en date la chaine de caractères de l API'''
    return datetime.strptime(chaine,'%Y%m%dT%H%M%S')

def convertir_en_chaine(dt) :
    ''' on convertit en chaîne de caractères un datetime'''
    return datetime.strftime(dt, '%Y%m%dT%H%M%S')

day = input("Entrer la date de départ (DD/MM/YYYY): ")
hour = input("Entrer une heure de départ (HH:Min):")

#convertir la date entrée en chaîne sous forme la forme de l'API
date_str = day + " " + hour

date_depart_str = datetime.strptime(date_str, '%d/%m/%Y %H:%M').strftime('%Y%m%dT%H%M%S')
date_depart_str2 = convertir_en_temps(date_depart_str)
date_depart = convertir_en_chaine(date_depart_str2)

print(date_depart)

gare_depart = 'stop_area:SNCF:87212027'

#connaitre les départs depuis la gare de Strasbourg
def destinations_possibles(origine, datetime) :
     return requests.get('https://api.sncf.com/v1/coverage/sncf/stop_areas/{}/' \
                        'departures?from_datetime={}'.format(origine, datetime) ,
                        auth=(token_auth, '')).json()

#connaitre les trajets depuis la gare de Strasbourg
def trajets_possibles(gare_depart, date_heure_depart, date_heure_max) :
    
    destinations = []

    # on interroge l'API tant qu'il renvoie des informations
    # sur les trains partant de Gare de Strasbourg

    while convertir_en_temps(date_heure_depart) < convertir_en_temps(date_heure_max) :
    # on prend toutes les destinations qui partent à partir d'une certaine heure
        destinations = destinations + destinations_possibles(
            gare_depart, date_heure_depart)['departures']

        nombre_resultats = destinations_possibles(
            gare_depart, date_heure_depart)['pagination']['items_on_page']

        if nombre_resultats <= 0:
            break

        # on trouve l'heure max de la première série de 10 solutions que l'application renvoie
        # on remplace l'heure qu'on cherche par celle là
        resultats = destinations_possibles(gare_depart, date_heure_depart)
        date_heure_depart = resultats['departures'][nombre_resultats-1][
            'stop_date_time']['departure_date_time']

    return destinations


# on trouve l'ensemble des trajets dont le départ est compris entre deux horaires
around = input("Quelle est l'heure maximum à laquelle vous pouvez partir (HH:Min):):")
date_heure_max_str = day + " " + around
date_heure_max_str2 = datetime.strptime(date_heure_max_str, '%d/%m/%Y %H:%M').strftime('%Y%m%dT%H%M%S')
date_depart2_str2 = convertir_en_temps(date_heure_max_str2)
date_max = convertir_en_chaine(date_depart2_str2)

print("entre", hour, "et", around)

# on demande à avoir tous les trajets partant de gare de Strasbourg
# entre deux heures précises
# on veut supprimer ceux pour lesquels le départ est après l'heure max

if date_depart2_str2 <= convertir_en_temps(date_max):
    destinations_depuis_Strasbourg = trajets_possibles(gare_depart, date_depart, date_max)
    # Afficher les informations sous forme de tableau 
    #Aligne le texte à gauche et occuper une largeur de 20 caractères. 
    print("{:<20}  {:<20}".format("Ville", "Heure"))
    tableau = []
    for depart in destinations_depuis_Strasbourg:
        heure_depart = convertir_en_temps(depart['stop_date_time']['departure_date_time']).time()
        if (date_max >= date_depart):
            tableau.append({
            'ville': depart['display_informations']['direction'],
            'heure': heure_depart
            })
    
    print("Je peux prendre", len(destinations_depuis_Strasbourg),
          "trains qui partent entre", hour, "et", around, "de Gare de Strasbourg")
else:
    print("L'heure maximum doit être supérieure ou égale à l'heure de départ", hour)
print(heure_depart)

df_tableau = pd.DataFrame(tableau)

# Affichage du tableau
df_tableau.head(40)

Entrer la date de départ (DD/MM/YYYY): 04/10/2023
Entrer une heure de départ (HH:Min):10:00
20231004T100000
Quelle est l'heure maximum à laquelle vous pouvez partir (HH:Min):):12:00
entre 10:00 et 12:00
Ville                 Heure               
Je peux prendre 40 trains qui partent entre 10:00 et 12:00 de Gare de Strasbourg
12:00:00


,ville,heure
0,Barr (Barr),10:00:00
1,Rennes (Rennes),10:01:00
2,Rothau (Rothau),10:06:00
3,Sélestat (Sélestat),10:07:00
4,Haguenau (Haguenau),10:10:00
5,Molsheim (Molsheim),10:18:00
6,Nancy (Nancy),10:19:00
7,Bâle - SBB (Basel),10:21:00
8,Mommenheim (Mommenheim),10:24:00
9,Niederbronn-les-Bains (Niederbronn-les-Bains),10:27:00


In [6]:
# =============================================================================
# Partie coordonnées
# =============================================================================
import json
import requests
# Entrer le nom de la gare d'arrivée 
ville = input("Entrer le nom de la gare d'arrivée que vous avez choisit:")

# Récupérer le fichier JSON
url = "https://geo.api.gouv.fr/communes"
params = {"nom": ville, "fields": "centre", "format": "json"}
response = requests.get(url, params=params)

# Extraire la latitude et la longitude
if response.status_code == 200:
    try:
        data = json.loads(response.text)
        latitude = data[0]["centre"]["coordinates"][1]
        longitude = data[0]["centre"]["coordinates"][0]
        print(f"Longitude : {longitude}, Latitude : {latitude}")
    except:
        print(f"Les coordonnées pour {ville} ne sont pas disponibles.")
else:
    print(f"Une erreur s'est produite lors de la récupération des coordonnées.")

Entrer le nom de la gare d'arrivée que vous avez choisit:Colmar
Longitude : 7.3924, Latitude : 48.1115


In [8]:
  
# =============================================================================
# Partie randonnée
# =============================================================================
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options

perimetre=input("Entrer la distance autour de la gare que tu souhaites en km:")

# Créer une instance de Options
options = Options()
options.headless = True

driver = webdriver.Firefox(executable_path=r'C:\Users\Marion\OneDrive\Documents\applications\webdriver\geckodriver.exe', options=options)
driver.get('https://www.komoot.com/fr-fr/discover/{}/@{},{}/tours?max_distance={}820&sport=hike'.format(ville, latitude, longitude, perimetre))
WebDriverWait(driver, 10).until(lambda driver: driver.find_elements(By.XPATH, "//div[@class='css-1m9b2rc']"))
time.sleep(3)
refuse_button = driver.find_element(By.XPATH, "//button[contains(@class, 'css-1y2vksk')]")
refuse_button.click()
time.sleep(2)
nom = driver.find_elements(By.XPATH, "//h4[@class='css-1t8cfg7']")
duree = driver.find_elements(By.XPATH, "//span[@class='tw-font-bold css-1u8qly9' and @data-test-id='t_duration_value']")
distance = driver.find_elements(By.XPATH,"//span[@class='tw-font-bold css-1u8qly9' and @data-test-id='t_distance_value']")

rando = []
for nom, duree, distance, in zip(nom, duree, distance):
    rando.append({
        'Nom': nom.text.strip(),
        'Duree': duree.text.strip(),
        'Distance': distance.text.strip(),        
    })

for r in rando:
    for k, v in r.items():
        if v == "":
            r[k] = "N/A"
            
def convertir_en_duree(chaine):
    return datetime.strptime(chaine, '%H:%M')

Temps_rando = input("Entre le temps maximum que tu souhaites que la randonnée dure (HH:Min):")
temps_rando = convertir_en_duree(Temps_rando)

randos_compatibles = []

for r in rando:
    duree_rando = convertir_en_duree(r['Duree'])
    if duree_rando <= temps_rando:
        randos_compatibles.append(r)
        
# Création du DataFrame
df_randos = pd.DataFrame(randos_compatibles)

# Tri par ordre croissant de la colonne "Duree"
df_randos = df_randos.sort_values('Duree', ascending=False)

# Affichage du tableau
print(df_randos.shape)
df_randos.head(30)

Entrer la distance autour de la gare que tu souhaites en km:3
Entre le temps maximum que tu souhaites que la randonnée dure (HH:Min):5:00


KeyError: 'Duree'

In [7]:
tableau = []

if date_depart2_str2 <= convertir_en_temps(date_max):
    destinations_depuis_Strasbourg = trajets_possibles(gare_depart, date_depart, date_max)
    print("{:<20}  {:<20}".format("Ville", "Heure"))
    for depart in destinations_depuis_Strasbourg:
        heure_depart = convertir_en_temps(depart['stop_date_time']['departure_date_time']).time()
        if (date_max >= date_depart):
            # on crée un dictionnaire avec les informations pour cette destination
            dict_destination = {
                "ville": depart['display_informations']['direction'],
                "heure": str(heure_depart)
            }
            # on ajoute le dictionnaire au tableau
            tableau.append(dict_destination)
            # on affiche les informations pour cette destination
            print("{:<20}  {:<20}".format(dict_destination["ville"], dict_destination["heure"]))
    
    print("Je peux prendre", len(destinations_depuis_Strasbourg), "trains qui partent entre", hour, "et", around, "de Gare de Strasbourg")
else:
    print("L'heure maximum doit être supérieure ou égale à l'heure de départ", hour)

# on crée le DataFrame à partir du tableau de dictionnaires
df_tableau = pd.DataFrame(tableau)

# on écrit le DataFrame dans un fichier CSV
df_tableau.to_csv("tableau.csv", index=False)

# on affiche le DataFrame dans la console
print(df_tableau.shape.to_string(index=False))
df_tableau.head

Ville                 Heure               
Barr (Barr)           10:00:00            
Rennes (Rennes)       10:01:00            
Rothau (Rothau)       10:06:00            
Sélestat (Sélestat)   10:07:00            
Haguenau (Haguenau)   10:10:00            
Molsheim (Molsheim)   10:18:00            
Nancy (Nancy)         10:19:00            
Bâle - SBB (Basel)    10:21:00            
Mommenheim (Mommenheim)  10:24:00            
Niederbronn-les-Bains (Niederbronn-les-Bains)  10:27:00            
Niederbronn-les-Bains (Niederbronn-les-Bains)  10:27:00            
Barr (Barr)           10:30:00            
Sélestat (Sélestat)   10:37:00            
Paris Est (Paris)     10:45:00            
Paris Est (Paris)     10:45:00            
Haguenau (Haguenau)   10:48:00            
Metz (Metz)           10:49:00            
Offenburg (Offenburg)  10:50:00            
Bâle - SBB (Basel)    10:51:00            
Saverne (Saverne)     10:54:00            
Saverne (Saverne)     10:54:00            


AttributeError: 'tuple' object has no attribute 'to_string'

In [6]:
import pandas as pd
df = pd.read_csv("tableau.csv")
# affichage interactif du DataFrame dans une fenêtre séparée
df.to_csv("tableau.csv")